# Introduction
This notebook is created during the hackathon on East Africa (March 2024).
It downloads an OSM network based on the combined extent of the various regions that are provided as input folders, containing a file `gis\region.geojson`.

## Determine the polygon of the total extent of the regions

In [10]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely import geometry, Polygon
from json import dump
from osgeo import gdal, osr
from shutil import copyfile

IMPORT_FOLDER = Path('P:\moonshot2-casestudy\SFINCS\models')
OUTPUT_FOLDER = Path('P:\moonshot2-casestudy\RA2CE')

In [6]:
# Loop over region folders and concatenate the extents
_combined_gdf = gpd.GeoDataFrame()
for _region in [f for f in IMPORT_FOLDER.iterdir() if f.is_dir()]:
    _extent_file = _region.joinpath('gis', 'region.geojson')
    assert _extent_file.is_file()
    _gdf = gpd.read_file(_extent_file)
    _combined_gdf = pd.concat([_combined_gdf, _gdf.to_crs(4326)], ignore_index=True)

# Extract polygon of the total extent spanning the concatenated regions
_total_extent = _combined_gdf.total_bounds
_polygon = Polygon(geometry.box(*_total_extent))

# Write the combined extent to a new file
_combined_extent_file = IMPORT_FOLDER.joinpath('combined_extent.geojson')
_combined_geojson = _combined_gdf.to_json()
with open(_combined_extent_file, 'w') as f:
    dump(_combined_geojson, f)



## Create network
Then network is created from a polygon based on the total extent of the regions.

In [9]:
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper

# Read network from polygon
_road_type_list = [RoadTypeEnum.MOTORWAY, RoadTypeEnum.MOTORWAY_LINK, RoadTypeEnum.TRUNK, RoadTypeEnum.TRUNK_LINK, RoadTypeEnum.PRIMARY, RoadTypeEnum.PRIMARY_LINK, RoadTypeEnum.SECONDARY, RoadTypeEnum.SECONDARY_LINK, RoadTypeEnum.TERTIARY, RoadTypeEnum.TERTIARY_LINK]
_network_section = NetworkSection(network_type=NetworkTypeEnum.DRIVE, road_types=_road_type_list, save_gpkg=True)
_config_data = NetworkConfigData(network=_network_section, static_path=IMPORT_FOLDER)
_network_wrapper = OsmNetworkWrapper.with_polygon(_config_data, _polygon)


c:\Users\klapwij\code\ra2ce\.env\lib\site-packages\osmnx\simplification.py:454: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = utils_graph.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\klapwij\code\ra2ce\.env\lib\site-packages\osmnx\simplification.py:501: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid


## Export the network to file

In [13]:
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter

# Export the graph to a GeoPackage
_exporter = MultiGraphNetworkExporter(basename='TotalNetwork', export_types=['gpkg'])
[_graph, _gdf] = _network_wrapper.get_network()
_exporter.export(export_path=Path(OUTPUT_FOLDER).joinpath('static'), export_data=_graph)


100%|██████████| 6275/6275 [00:00<00:00, 87750.75it/s]


## Prepare floodmaps

In [6]:
# Reproject and save the floodmaps
_output_folder = OUTPUT_FOLDER.joinpath('floodmaps')
if not _output_folder.exists():
    _output_folder.mkdir(parents=True)
    
for _region in [f for f in IMPORT_FOLDER.iterdir() if f.is_dir()]:
    _models_dir = _region.joinpath('floodmaps')
    for _floodmap in _models_dir.iterdir():
        _output_file = _output_folder.joinpath(_floodmap.name)
        
        _input_ds = gdal.Open(str(_floodmap))
        _source_proj = _input_ds.GetProjection()
        _srs = osr.SpatialReference(wkt=_source_proj)
        if _srs.IsProjected:
            _source_cs = _srs.GetAttrValue('PROJCS')
            gdal.Warp(str(_output_file), str(_floodmap), srcSRS=_source_cs, dstSRS='EPSG:4326')
        else:
            copyfile(_floodmap, _output_file)
        break
    break

AttributeError: 'str' object has no attribute 'GetAttrValue'